In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df= pd.read_csv('../../DataSets/Restaurant_Reviews.tsv', sep='\t')

In [3]:
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


# Text preprocessing

# Same Case- i.e. convert whole text into lover case 

In [4]:
review= 'Wow... Loved this place.'
review.lower()


'wow... loved this place.'

# Removing Unwanted Charecters from Text

In [5]:
import re

In [6]:
review= 'Wow... Loved this place.'
review= review.lower()
re.sub('[^a-z]',' ',review)


'wow    loved this place '

In [7]:
review= 'Wow... Loved this place.'
review= review.lower()
review= re.sub('[^a-z]',' ',review)
review.split()


['wow', 'loved', 'this', 'place']

In [8]:
review= 'Wow... Loved this place.'
review= review.lower()
review= re.sub('[^a-z]',' ',review)
words= review.split()


# Removing Stopwords

In [9]:
import nltk

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pavilion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stopwords= nltk.corpus.stopwords.words('english')    # List of Stopwords

In [12]:
stopwords.remove('not')

In [13]:
review= 'Wow... Loved this place.'
review= review.lower()
review= re.sub('[^a-z]',' ',review)
words= review.split()
final_words= [word for word in words if not word in stopwords]
final_words

['wow', 'loved', 'place']

# Stemming

In [14]:
stemmer = nltk.stem.PorterStemmer()     # Stemming
stemmer.stem('running')

'run'

In [15]:
review= 'Wow... Loved this place.'
review= review.lower()
review= re.sub('[^a-z]',' ',review)
words= review.split()
final_words= [stemmer.stem(word) for word in words if not word in stopwords]
' '.join(final_words)

'wow love place'

In [16]:
# Defining User Define Func

In [17]:
def preprocess(review):
    review= review.lower()
    review= re.sub('[^a-z]',' ',review)
    words= review.split()
    final_words= [stemmer.stem(word) for word in words if not word in stopwords]
    return ' '.join(final_words)

In [18]:
preprocess('Not tasty and the texture was just nasty.')

'not tasti textur nasti'

In [19]:
preprocess('Crust is not good.')    # as not is in stopwords it is predicting wrong

'crust not good'

In [20]:
# Apply function to DF

In [21]:
df['Review'].apply(preprocess)    # Preprocessing Done

0                                         wow love place
1                                         crust not good
2                                 not tasti textur nasti
3      stop late may bank holiday rick steve recommen...
4                                select menu great price
                             ...                        
995                        think food flavor textur lack
996                               appetit instantli gone
997                 overal not impress would not go back
998    whole experi underwhelm think go ninja sushi n...
999    wast enough life pour salt wound draw time too...
Name: Review, Length: 1000, dtype: object

In [22]:
df['processed_review']= df['Review'].apply(preprocess)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
vectorizer = CountVectorizer()
vectorizer.fit(df['processed_review'])

CountVectorizer()

In [25]:
len(vectorizer.get_feature_names())

1566

# Bag OF Words Model

In [26]:
bow_table= vectorizer.transform(df['processed_review'])

In [27]:
bow_table   # In the form of sparse metrix ... so need to convert it into array

<1000x1566 sparse matrix of type '<class 'numpy.int64'>'
	with 5484 stored elements in Compressed Sparse Row format>

In [28]:
bow_table.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
X_train= bow_table.toarray()
y_yrain= df['Liked']

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
model= RandomForestClassifier(n_estimators=90,max_depth=3,min_samples_leaf=2)
model.fit(X_train,y_yrain)

RandomForestClassifier(max_depth=3, min_samples_leaf=2, n_estimators=90)

In [36]:
def predict_review(review):
    review=preprocess(review)
    X_test=vectorizer.transform([review])
    return model.predict(X_test)

In [37]:
predict_review('Not tasty and the texture was just nasty.')

array([0], dtype=int64)

In [42]:
predict_review('food wasnt good')

array([1], dtype=int64)